In [5]:
import SimpleITK as sitk
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed
from IPython.display import clear_output
import pprint
import os, sys
import pandas_profiling
import sys
import json
from scipy import ndimage
import nibabel as nib 
from pathlib import Path
sys.path.insert(0, str(Path(r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\src\registration_metric.py').resolve().parent.parent))
from src.eda_nifti import *
from src.registration_metric import *

In [ ]:
nifti_anal =NiftiAnalysis()

In [ ]:
task1_nifti_files_list = nifti_anal.recursive_find_all_files(r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\data\task1', '.nii.gz')

In [ ]:
raw_path = r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\data\task1\rawdata\sub-strokecase0001\ses-0001\sub-strokecase0001_ses-0001_dwi.nii.gz'

In [ ]:
mask_path = r'C:\Users\parkm\Desktop\github\analysis_nifti_for_deeplearning\data\task1\derivatives\sub-strokecase0001\ses-0001\sub-strokecase0001_ses-0001_msk.nii.gz'

# 1. 원본 단면 확인

In [ ]:
raw_nii = nib.load(raw_path)
raw_nii_array = raw_nii.get_fdata()
raw_nii_array.shape
plt.imshow(raw_nii_array[:,:,49])

# 2. 원본 + n4 correction 확인

In [ ]:
input = sitk.ReadImage(raw_path)
image = sitk.Cast(input, sitk.sitkFloat32)
# print(type(image))
correctedImg = sitk.N4BiasFieldCorrection(image)
sitk.WriteImage(correctedImg, 'test.nii.gz') # outfile is just string variable or path

In [ ]:
nii = nib.load('test.nii.gz')
nii_array = nii.get_fdata()
nii_array.shape

In [ ]:
plt.imshow(nii_array[:,:,49])

# 3. 원본에서 라벨 마스크 아웃 + n4 correction
- antspy로 진행하면 antsimage로 작업하고 바로 저장하면되는데, 리눅스 환경 x, 대충 몇개만 돌려서 확인해보는 용도

In [ ]:
raw_nii = nib.load(raw_path)
raw_nii_array = raw_nii.get_fdata()
raw_affine = raw_nii.affine

In [ ]:
mask_nii = nib.load(mask_path)

In [ ]:
mask_nii_array = mask_nii.get_fdata()

In [ ]:
raw_nii_array.shape

In [ ]:
mask_nii_array.shape

In [ ]:
plt.imshow(mask_nii_array[:,:,49])

In [ ]:
np.unique(np.ravel(raw_nii_array[:,:,49]))

In [ ]:
np.unique(np.ravel(mask_nii_array[:,:,49]))

In [ ]:
raw_max_level_pixel = np.ravel(raw_nii_array).max()

In [ ]:
mask_out_array = raw_nii_array + (mask_nii_array * raw_max_level_pixel*100)

In [ ]:
mask_out_array[mask_out_array>raw_max_level_pixel+5] = 0

In [ ]:
np.unique(np.ravel(mask_out_array[:,:,49]))

In [ ]:
np.unique(np.ravel(mask_out_array[:,:,49]))

In [ ]:
plt.imshow(mask_out_array[:,:,49])

In [ ]:
img = nib.Nifti1Image(mask_out_array, raw_affine) 
img.to_filename('raw_mask_out.nii.gz')

In [ ]:
input = sitk.ReadImage('raw_mask_out.nii.gz')
image = sitk.Cast(input, sitk.sitkFloat32)
# print(type(image))
correctedImg = sitk.N4BiasFieldCorrection(image)
sitk.WriteImage(correctedImg, 'n4_raw_mask_out.nii.gz')

In [ ]:
## Segmentation된 부분만 확인

In [ ]:
mask = np.any(mask_nii_array.astype(bool), axis=-1)

In [ ]:
masked_img = raw_nii_array[:,:,49] * mask_nii_array[:,:,49]

In [ ]:
masked_img

In [ ]:
plt.imshow(masked_img)

In [ ]:
np.unique(np.ravel(masked_img))

In [ ]:
plt.imshow(raw_nii_array[:,:,49])

In [ ]:
plt.imshow(mask_nii_array[:,:,49])

In [ ]:
np.unique(np.ravel(mask_nii_array[:,:,49]))

In [7]:
def n4_mask_out_n4_correction(input_raw_nifti_path: str, input_mask_nifti_path: str, n4_correction_result_save_path: str) ->np.array:
    
    raw_nii = nib.load(input_raw_nifti_path)
    raw_nii_array = raw_nii.get_fdata()
    raw_affine = raw_nii.affine
    raw_max_level_pixel = np.ravel(raw_nii_array).max()
    
    mask_nii = nib.load(input_mask_nifti_path)
    mask_nii_array = mask_nii.get_fdata()
    assert 1 in mask_nii_array or 0 in mask_nii_array, "incorrect values in mask array(mask array must be include only 0 or 1)" 
    
    
    mask_out_array = raw_nii_array * (1-mask_nii_array)
    
    img = nib.Nifti1Image(mask_out_array, raw_affine) 
    img.to_filename('temp_raw_mask_out.nii.gz')
    
    input = sitk.ReadImage('temp_raw_mask_out.nii.gz')
    image = sitk.Cast(input, sitk.sitkFloat32)
    # print(type(image))
    n4_correction_on_not_mask_region_only = sitk.N4BiasFieldCorrection(image)
    sitk.WriteImage(n4_correction_on_not_mask_region_only, os.path.join(n4_correction_result_save_path,output_raw_nifti_path+'/n4_correction_on_not_mask_region_only.nii.gz'))
    
    return n4_correction_on_not_mask_region_only

In [6]:
def n4_mask_out_n4_correction_and_merge_with_lesion(input_raw_nifti_path: str, input_mask_nifti_path: str, n4_correction_result_save_path: str) ->np.array:
    
    raw_nii = nib.load(input_raw_nifti_path)
    raw_nii_array = raw_nii.get_fdata()
    raw_affine = raw_nii.affine
    raw_max_level_pixel = np.ravel(raw_nii_array).max()
    
    mask_nii = nib.load(input_mask_nifti_path)
    ## 효과가 있다면, 이 부분에서 voxel erosion 추가해서 라벨 근처 확장시켜주면 좋을 것 같습니다.
    mask_nii_array = mask_nii.get_fdata()
    assert 1 in mask_nii_array or 0 in mask_nii_array, "incorrect values in mask array(mask array must be include only 0 or 1)" 
    
    
    mask_out_array = raw_nii_array * (1-mask_nii_array)
    
    img = nib.Nifti1Image(mask_out_array, raw_affine) 
    img.to_filename('temp_raw_mask_out.nii.gz')
    
    input = sitk.ReadImage('temp_raw_mask_out.nii.gz')
    image = sitk.Cast(input, sitk.sitkFloat32)
    # print(type(image))
    correctedImg_without_legion = sitk.N4BiasFieldCorrection(image)
    
    mask_legion_of_raw_data = raw_nii_array * mask_nii_array
    n4_correction_on_not_mask_region_only = correctedImg_without_legion + mask_legion_of_raw_data
    
    sitk.WriteImage(n4_correction_on_not_mask_region_only, os.path.join(n4_correction_result_save_path,output_raw_nifti_path+'/correctedImg_without_legion.nii.gz'))
    
    
    
    return correctedImg_without_legion 